# Header

In [1]:
import numpy as np
from numpy import array, newaxis, expand_dims
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy as copy
import matplotlib.backends.backend_pdf as pdflib
import scipy as sp
import scipy.stats as stats
import math
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_predict
# plotly.tools.set_credentials_file(username='mhossa10', api_key='CSWpOV3xHUVRzimjfHSB')
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Color Lists

In [2]:
color_list1 = ['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e', '#e6ab02']
color_list2 = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33']
color_list3 = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c']

# Test Matrices

In [3]:
# test_matrices = [
#     "delaunay_n20"
#     , "NACA0015"
# #     , "belgium.osm"
#     , "AS365"
# #     , "roadNet-TX"
#     , "road_central"
#     , "NLR"
#     , "hugetrace-00010"
#     , "nlpkkt200"
# #     , "uk-2002"
# ]

test_matrices = [
#     "delaunay_n20"
#     , "NACA0015"
#     , "belgium"
     "AS365"
#     , "roadNet-TX"
    , "road_central"
    , "NLR"
    , "hugetrace-00010"
#     , "nlpkkt200"
#     , "nlpkkt240"
#     , "nlpkkt160"
    , "uk2002"
    , "uk-2002"
    , "333SP"
    , "M6"
#     , "af_shell10" 
#     , "cage15" 
#     , "in-2004" 
#     , "kkt_power" 
    , "road_usa"
    
#     , "asia" 
#     , "aumikw1" 
#     , "com-lj" 
#     , "delaunay_n21" 
#     , "delaunay_n22" 
#     , "delaunay_n23" 
#     , "delaunay_n24" 
#     , "europe" 
#     , "germany" 
#     , "great-britain" 
#     , "hugebubbles-00000" 
#     , "hugebubbles-00010" 
#     , "hugebubbles-00020" 
#     , "hugetrace-00000" 
#     , "hugetrace-00020" 
#     , "hugetric-00000" 
#     , "hugetric-00020" 
#     , "italy" 
#     , "ldoor"
]

random_test_matrices = [
#     "delaunay_n20_random"
#     , "NACA0015_random"
#     , "belgium_random"
     "AS365_random"
    , "roadNet-TX_random"
    , "road_central_random"
    , "NLR_random"
    , "hugetrace-00010_random"
#     , "nlpkkt200_random"
#     , "nlpkkt240_random"
#     , "nlpkkt160_random"
    , "uk2002_random"
    , "uk-2002_random"
    , "333SP_random"
    , "M6_random"
#     , "af_shell10_random" 
#     , "cage15_random" 
#     , "in-2004_random" 
#     , "kkt_power_random" 
    , "road_usa_random"
    
#     , "asia_random" 
#     , "aumikw1_random" 
#     , "com-lj_random" 
#     , "delaunay_n21_random" 
#     , "delaunay_n22_random" 
#     , "delaunay_n23_random" 
#     , "delaunay_n24_random" 
#     , "europe_random" 
#     , "germany_random" 
#     , "great-britain_random" 
#     , "hugebubbles-00000_random" 
#     , "hugebubbles-00010_random" 
#     , "hugebubbles-00020_random" 
#     , "hugetrace-00000_random" 
#     , "hugetrace-00020_random" 
#     , "hugetric-00000_random" 
#     , "hugetric-00020_random" 
#     , "italy_random" 
#     , "ldoor_random"
]

random_to_org_map = {
    "delaunay_n20_random"        : "delaunay_n20"
    , "NACA0015_random"          : "NACA0015"
    , "belgium_random"           : "belgium"
    , "AS365_random"             : "AS365"
    , "roadNet-TX_random"        : "roadNet-TX"
    , "road_central_random"      : "road_central"
    , "NLR_random"               : "NLR"
    , "hugetrace-00010_random"   : "hugetrace-00010"
#     , "nlpkkt200_random"         : "nlpkkt200"
#     , "nlpkkt240_random"         : "nlpkkt240"
#     , "nlpkkt160_random"         : "nlpkkt160"
    , "uk2002_random"            : "uk2002"
    , "uk-2002_random"           : "uk-2002"
    , "333SP_random"             : "333SP"
    , "M6_random"                : "M6"
#     , "af_shell10_random"        : "af_shell10"
#     , "cage15_random"            : "cage15"
#     , "in-2004_random"           : "in-2004"
    , "kkt_power_random"         : "kkt_power"
    , "road_usa_random"          : "road_usa"
    
#     , "asia_random"              : "asia"
#     , "aumikw1_random"           : "aumikw1"
#     , "com-lj_random"            : "com-lj"
#     , "delaunay_n21_random"      : "delaunay_n21"
#     , "delaunay_n22_random"      : "delaunay_n22" 
#     , "delaunay_n23_random"      : "delaunay_n23"
#     , "delaunay_n24_random"      : "delaunay_n24"
#     , "europe_random"            : "europe"
#     , "germany_random"           : "germany"
#     , "great-britain_random"     : "great-britain"
#     , "hugebubbles-00000_random" : "hugebubbles-00000" 
#     , "hugebubbles-00010_random" : "hugebubbles-00010"
#     , "hugebubbles-00020_random" : "hugebubbles-00020"
#     , "hugetrace-00000_random"   : "hugetrace-00000"
#     , "hugetrace-00020_random"   : "hugetrace-00020"
#     , "hugetric-00000_random"    : "hugetric-00000"
#     , "hugetric-00020_random"    : "hugetric-00020"
#     , "italy_random"             : "italy"
#     , "ldoor_random"             : "ldoor"
}


rmat_test_matrices = [
    "rmat_100M2M"
    , "rmat_100M3M"
    , "rmat_200M4M"
    , "rmat_500M3M"
    , "rmat_500M4M"
]

# Dynamic CSR Local SpMV(LK-SpMV)

In [7]:
csr_dyn_lk_spmv_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
m_size = []
csr_dyn_lk_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_Local_KWAY_CSR_SpMV_Comm.csv")
csr_dyn_lk_spmv = csr_dyn_lk_spmv[~csr_dyn_lk_spmv['Name'].str.contains("rmat")].reset_index(drop=True)
csr_dyn_lk_spmv = csr_dyn_lk_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index()
csr_dyn_lk_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
csr_dyn_lk_spmv = csr_dyn_lk_spmv.reset_index()
csr_dyn_lk_spmv = csr_dyn_lk_spmv.sort_values(by=["AvgRows", "AvgNPR"]).reset_index(drop=True)

density = []
min_time_factor = 1.0
if min(csr_dyn_lk_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(csr_dyn_lk_spmv["AvgTime"])
for i,r in csr_dyn_lk_spmv.iterrows():
    csr_dyn_lk_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["AvgNNZ"])/float(r["AvgRows"] * r["AvgRows"]))
csr_dyn_lk_spmv["Density"] = density

column = "AvgTime"
csr_dyn_lk_spmv_train_data = csr_dyn_lk_spmv[~csr_dyn_lk_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
csr_dyn_lk_spmv_test_data = csr_dyn_lk_spmv[csr_dyn_lk_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

#     csr_local_kway_train = csr_local_kway_train_data[["AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()
#     csr_local_kway_test = csr_local_kway_test_data[["AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

csr_dyn_lk_spmv_train = csr_dyn_lk_spmv_train_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgLocalNNZ", "AvgGlobalNNZ", "AvgInterProcessCall", "AvgDataSend", "AvgNPRSD", "Density"]].to_numpy()
csr_dyn_lk_spmv_test = csr_dyn_lk_spmv_test_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgLocalNNZ", "AvgGlobalNNZ", "AvgInterProcessCall", "AvgDataSend", "AvgNPRSD", "Density"]].to_numpy()

csr_dyn_lk_spmv_X = np.concatenate((csr_dyn_lk_spmv_train, csr_dyn_lk_spmv_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
csr_dyn_lk_spmv_X = scaler.fit_transform(csr_dyn_lk_spmv_X)
X = csr_dyn_lk_spmv_X[0:len(csr_dyn_lk_spmv_train)]
csr_dyn_lk_spmv_test_X = csr_dyn_lk_spmv_X[len(csr_dyn_lk_spmv_train):]
train_y = np.array(csr_dyn_lk_spmv_train_data[column])
test_y = np.array(csr_dyn_lk_spmv_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,6e4, num=50):
#     for e in np.linspace(0.0001, 0.9, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:  {'C': 1244.0816326530612, 'epsilon': 0.9}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(csr_dyn_lk_spmv_test_X)

max_err = 0
avg_err = 0
for i,row in csr_dyn_lk_spmv_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")           
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRows"])
    nnz_r.append(row["AvgNPR"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("CSR LK-SPMV")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/csr_dyn_lk_spmv_test_data.shape[0])
    
csr_dyn_lk_spmv_pred["Name"] = mat
csr_dyn_lk_spmv_pred["Model"] = spmv_model
csr_dyn_lk_spmv_pred["Nodes"] = node
csr_dyn_lk_spmv_pred["Process"] = prcs
csr_dyn_lk_spmv_pred["Matrix Size"] = m_size
csr_dyn_lk_spmv_pred["Avg Row"] = rows
csr_dyn_lk_spmv_pred["Nonzero per Row"] = nnz_r
csr_dyn_lk_spmv_pred["Actual Time"] = a_time
csr_dyn_lk_spmv_pred["Predicted Time"] = p_time
csr_dyn_lk_spmv_pred["Error"] = err

M6 0.82848 & 0.62033 & 25.12\% \\ \hline
333SP 0.82442 & 0.62124 & 24.65\% \\ \hline
AS365 0.85384 & 0.65935 & 22.78\% \\ \hline
M6 0.87088 & 0.66295 & 23.88\% \\ \hline
NLR 0.94891 & 0.71506 & 24.64\% \\ \hline
333SP 0.83798 & 0.66778 & 20.31\% \\ \hline
AS365 0.92311 & 0.7089 & 23.21\% \\ \hline
NLR 0.98853 & 0.77053 & 22.05\% \\ \hline
M6 1.0582 & 0.85682 & 19.03\% \\ \hline
333SP 1.0534 & 0.86781 & 17.62\% \\ \hline
AS365 1.1445 & 0.91949 & 19.66\% \\ \hline
M6 1.2529 & 0.9931 & 20.74\% \\ \hline
NLR 1.2696 & 1.0009 & 21.16\% \\ \hline
333SP 1.1968 & 1.0078 & 15.8\% \\ \hline
AS365 1.3325 & 1.0638 & 20.16\% \\ \hline
NLR 1.4254 & 1.1655 & 18.23\% \\ \hline
hugetrace-00010 1.6409 & 1.5094 & 8.016\% \\ \hline
hugetrace-00010 1.7719 & 1.6648 & 6.043\% \\ \hline
road_central 1.7799 & 1.5837 & 11.02\% \\ \hline
road_central 1.933 & 1.7559 & 9.166\% \\ \hline
hugetrace-00010 2.1755 & 2.1832 & 0.3577\% \\ \hline
road_central 2.3672 & 2.3214 & 1.934\% \\ \hline
hugetrace-00010 2.5408 & 2.5

# Dynamic COO Local SpMV(LK-SpMV)

In [8]:
coo_dyn_lk_spmv_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
m_size = []
coo_dyn_lk_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_Local_KWAY_COO_SpMV_Comm.csv")
coo_dyn_lk_spmv = coo_dyn_lk_spmv[~coo_dyn_lk_spmv['Name'].str.contains("rmat")].reset_index(drop=True)
coo_dyn_lk_spmv = coo_dyn_lk_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index()
coo_dyn_lk_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
coo_dyn_lk_spmv = coo_dyn_lk_spmv.reset_index()
coo_dyn_lk_spmv = coo_dyn_lk_spmv.sort_values(by=["AvgRows", "AvgNPR"]).reset_index(drop=True)

density = []
min_time_factor = 1.0
if min(coo_dyn_lk_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(coo_dyn_lk_spmv["AvgTime"])
for i,r in coo_dyn_lk_spmv.iterrows():
    coo_dyn_lk_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["AvgNNZ"])/float(r["AvgRows"] * r["AvgRows"]))
coo_dyn_lk_spmv["Density"] = density

column = "AvgTime"
coo_dyn_lk_spmv_train_data = coo_dyn_lk_spmv[~coo_dyn_lk_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
coo_dyn_lk_spmv_test_data = coo_dyn_lk_spmv[coo_dyn_lk_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

#     coo_local_kway_train = coo_local_kway_train_data[["AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()
#     coo_local_kway_test = coo_local_kway_test_data[["AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

coo_dyn_lk_spmv_train = coo_dyn_lk_spmv_train_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgLocalNNZ", "AvgGlobalNNZ", "AvgInterProcessCall", "AvgDataSend", "AvgNPRSD", "Density"]].to_numpy()
coo_dyn_lk_spmv_test = coo_dyn_lk_spmv_test_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgLocalNNZ", "AvgGlobalNNZ", "AvgInterProcessCall", "AvgDataSend", "AvgNPRSD", "Density"]].to_numpy()

coo_dyn_lk_spmv_X = np.concatenate((coo_dyn_lk_spmv_train, coo_dyn_lk_spmv_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
coo_dyn_lk_spmv_X = scaler.fit_transform(coo_dyn_lk_spmv_X)
X = coo_dyn_lk_spmv_X[0:len(coo_dyn_lk_spmv_train)]
coo_dyn_lk_spmv_test_X = coo_dyn_lk_spmv_X[len(coo_dyn_lk_spmv_train):]
train_y = np.array(coo_dyn_lk_spmv_train_data[column])
test_y = np.array(coo_dyn_lk_spmv_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,6e4, num=50):
#     for e in np.linspace(0.0001, 0.9, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:  {'C': 1244.0816326530612, 'epsilon': 0.9}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(coo_dyn_lk_spmv_test_X)

max_err = 0
avg_err = 0
for i,row in coo_dyn_lk_spmv_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")           
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRows"])
    nnz_r.append(row["AvgNPR"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("COO LK-SPMV")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/coo_dyn_lk_spmv_test_data.shape[0])
    
coo_dyn_lk_spmv_pred["Name"] = mat
coo_dyn_lk_spmv_pred["Model"] = spmv_model
coo_dyn_lk_spmv_pred["Nodes"] = node
coo_dyn_lk_spmv_pred["Process"] = prcs
coo_dyn_lk_spmv_pred["Matrix Size"] = m_size
coo_dyn_lk_spmv_pred["Avg Row"] = rows
coo_dyn_lk_spmv_pred["Nonzero per Row"] = nnz_r
coo_dyn_lk_spmv_pred["Actual Time"] = a_time
coo_dyn_lk_spmv_pred["Predicted Time"] = p_time
coo_dyn_lk_spmv_pred["Error"] = err

M6 0.82172 & 0.64796 & 21.15\% \\ \hline
333SP 0.83806 & 0.64834 & 22.64\% \\ \hline
AS365 0.84252 & 0.6897 & 18.14\% \\ \hline
M6 0.91517 & 0.69752 & 23.78\% \\ \hline
NLR 0.96953 & 0.74882 & 22.76\% \\ \hline
333SP 0.88583 & 0.70117 & 20.85\% \\ \hline
AS365 0.93303 & 0.74638 & 20\% \\ \hline
NLR 1.012 & 0.81193 & 19.77\% \\ \hline
M6 1.1229 & 0.8939 & 20.4\% \\ \hline
333SP 1.1087 & 0.90098 & 18.73\% \\ \hline
AS365 1.1667 & 0.96026 & 17.7\% \\ \hline
M6 1.278 & 1.0378 & 18.79\% \\ \hline
NLR 1.2909 & 1.0467 & 18.92\% \\ \hline
333SP 1.2385 & 1.046 & 15.54\% \\ \hline
AS365 1.3751 & 1.112 & 19.13\% \\ \hline
NLR 1.4365 & 1.2205 & 15.04\% \\ \hline
hugetrace-00010 1.6175 & 1.4928 & 7.712\% \\ \hline
hugetrace-00010 1.7601 & 1.6491 & 6.304\% \\ \hline
road_central 1.6137 & 1.5539 & 3.706\% \\ \hline
road_central 1.7274 & 1.7178 & 0.5513\% \\ \hline
hugetrace-00010 2.2015 & 2.1484 & 2.41\% \\ \hline
road_central 2.145 & 2.242 & 4.523\% \\ \hline
hugetrace-00010 2.5926 & 2.5121 & 3.105\

# Dynamic CSR GK-SpMV

In [31]:
csr_dyn_gk_spmv_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
m_size = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
csr_kway_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_KWAY_SpMV.csv")
csr_kway_spmv = csr_kway_spmv[~csr_kway_spmv['Name'].str.contains("rmat")].reset_index()
csr_kway_spmv = csr_kway_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index(drop=True)

csr_kway_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

csr_kway_spmv = csr_kway_spmv.sort_values(by=["AvgRows", "AvgNPR"]).reset_index(drop=True)
density = []
min_time_factor = 1.0
if min(csr_kway_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(csr_kway_spmv["AvgTime"])
for i,r in csr_kway_spmv.iterrows():
    csr_kway_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["AvgNNZ"])/float(r["AvgRows"] * r["AvgRows"]))
csr_kway_spmv["Density"] = density
column = "AvgTime"
csr_kway_train_data = csr_kway_spmv[~csr_kway_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
csr_kway_test_data = csr_kway_spmv[csr_kway_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

csr_kway_train = csr_kway_train_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

csr_kway_test = csr_kway_test_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

csr_kway_X = np.concatenate((csr_kway_train, csr_kway_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
csr_kway_X = scaler.fit_transform(csr_kway_X)
X = csr_kway_X[0:len(csr_kway_train)]
csr_kway_test_X = csr_kway_X[len(csr_kway_train):]
train_y = np.array(csr_kway_train_data[column])
test_y = np.array(csr_kway_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,6e4, num=10):
#     for e in np.linspace(0.0001, 0.1, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:{'C': 60000.0, 'epsilon': 0.1}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(csr_kway_test_X)

max_err = 0
avg_err = 0
for i,row in csr_kway_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], "&", r["Nodes"], "&", r["nProcess"], "&", '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")    
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRows"])
    nnz_r.append(row["AvgNPR"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("CSR GK-SPMV")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/csr_kway_test_data.shape[0])
csr_dyn_gk_spmv_pred["Name"] = mat
csr_dyn_gk_spmv_pred["Model"] = spmv_model
csr_dyn_gk_spmv_pred["Nodes"] = node
csr_dyn_gk_spmv_pred["Process"] = prcs
csr_dyn_gk_spmv_pred["Matrix Size"] = m_size
csr_dyn_gk_spmv_pred["Avg Row"] = rows
csr_dyn_gk_spmv_pred["Nonzero per Row"] = nnz_r
csr_dyn_gk_spmv_pred["Actual Time"] = a_time
csr_dyn_gk_spmv_pred["Predicted Time"] = p_time
csr_dyn_gk_spmv_pred["Error"] = err

M6 & 4 & 144 & 22.31 & 21.125 & 5.309\% \\ \hline
333SP & 4 & 144 & 23.632 & 22.366 & 5.358\% \\ \hline
AS365 & 4 & 144 & 24.177 & 22.874 & 5.385\% \\ \hline
M6 & 4 & 144 & 18.172 & 19.931 & 9.676\% \\ \hline
NLR & 4 & 144 & 26.413 & 25.02 & 5.276\% \\ \hline
333SP & 4 & 144 & 19.463 & 21.146 & 8.647\% \\ \hline
AS365 & 4 & 144 & 19.748 & 21.646 & 9.611\% \\ \hline
NLR & 4 & 144 & 21.457 & 23.749 & 10.68\% \\ \hline
M6 & 4 & 144 & 19.067 & 21.093 & 10.63\% \\ \hline
333SP & 4 & 144 & 19.998 & 22.387 & 11.95\% \\ \hline
AS365 & 4 & 144 & 20.554 & 22.919 & 11.51\% \\ \hline
M6 & 4 & 144 & 24.721 & 23.925 & 3.219\% \\ \hline
NLR & 4 & 144 & 22.428 & 25.157 & 12.17\% \\ \hline
333SP & 4 & 144 & 26.168 & 25.354 & 3.111\% \\ \hline
AS365 & 4 & 144 & 26.738 & 25.94 & 2.985\% \\ \hline
NLR & 4 & 144 & 29.247 & 28.412 & 2.854\% \\ \hline
hugetrace-00010 & 4 & 144 & 73.336 & 70.894 & 3.329\% \\ \hline
hugetrace-00010 & 4 & 144 & 63.688 & 68.97 & 8.293\% \\ \hline
road_central & 4 & 144 & 85.719 

# Dynamic COO GK-SpMV

In [21]:
coo_dyn_gk_spmv_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
m_size = []

coo_kway_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_KWAY_COO_SpMV.csv")
coo_kway_spmv = coo_kway_spmv[~coo_kway_spmv['Name'].str.contains("rmat")].reset_index(drop=True)
coo_kway_spmv = coo_kway_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index()
coo_kway_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
coo_kway_spmv = coo_kway_spmv.reset_index()
coo_kway_spmv = coo_kway_spmv.sort_values(by=["AvgRows", "AvgNPR"]).reset_index(drop=True)

density = []
min_time_factor = 1.0
if min(coo_kway_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(coo_kway_spmv["AvgTime"])
for i,r in coo_kway_spmv.iterrows():
    coo_kway_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["AvgNNZ"])/float(r["AvgRows"] * r["AvgRows"]))
coo_kway_spmv["Density"] = density

column = "AvgTime"
coo_kway_train_data = coo_kway_spmv[~coo_kway_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
coo_kway_test_data = coo_kway_spmv[coo_kway_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

coo_kway_train = coo_kway_train_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

coo_kway_test = coo_kway_test_data[["Nodes", "nProcess", "AvgRows", "AvgNNZ", "AvgNPR", "AvgNPRSD", "Density"]].to_numpy()

coo_kway_X = np.concatenate((coo_kway_train, coo_kway_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
coo_kway_X = scaler.fit_transform(coo_kway_X)
X = coo_kway_X[0:len(coo_kway_train)]
coo_kway_test_X = coo_kway_X[len(coo_kway_train):]
train_y = np.array(coo_kway_train_data[column])
test_y = np.array(coo_kway_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,6e4, num=50):
#     for e in np.linspace(0.0001, 0.9, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:  {'C': 5000.0, 'epsilon': 0.1}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(coo_kway_test_X)

max_err = 0
avg_err = 0
for i,row in coo_kway_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")           
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRows"])
    nnz_r.append(row["AvgNPR"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("COO GK-SPMV")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/coo_kway_test_data.shape[0])
coo_dyn_gk_spmv_pred["Name"] = mat
coo_dyn_gk_spmv_pred["Model"] = spmv_model
coo_dyn_gk_spmv_pred["Nodes"] = node
coo_dyn_gk_spmv_pred["Process"] = prcs
coo_dyn_gk_spmv_pred["Matrix Size"] = m_size
coo_dyn_gk_spmv_pred["Avg Row"] = rows
coo_dyn_gk_spmv_pred["Nonzero per Row"] = nnz_r
coo_dyn_gk_spmv_pred["Actual Time"] = a_time
coo_dyn_gk_spmv_pred["Predicted Time"] = p_time
coo_dyn_gk_spmv_pred["Error"] = err

M6 25.665 & 24.817 & 3.306\% \\ \hline
333SP 27.123 & 26.266 & 3.162\% \\ \hline
AS365 27.8 & 26.869 & 3.349\% \\ \hline
M6 22.977 & 24.366 & 6.043\% \\ \hline
NLR 30.396 & 29.406 & 3.259\% \\ \hline
333SP 24.324 & 25.834 & 6.206\% \\ \hline
AS365 25.025 & 26.453 & 5.707\% \\ \hline
NLR 27.352 & 29.033 & 6.143\% \\ \hline
M6 25.221 & 26.845 & 6.439\% \\ \hline
333SP 26.71 & 28.46 & 6.555\% \\ \hline
AS365 27.348 & 29.148 & 6.584\% \\ \hline
M6 31.16 & 30.068 & 3.505\% \\ \hline
NLR 29.959 & 31.999 & 6.809\% \\ \hline
333SP 32.563 & 31.835 & 2.237\% \\ \hline
AS365 33.306 & 32.588 & 2.154\% \\ \hline
NLR 36.468 & 35.712 & 2.073\% \\ \hline
hugetrace-00010 79.21 & 77.609 & 2.022\% \\ \hline
hugetrace-00010 70.32 & 77.47 & 10.17\% \\ \hline
road_central 91.204 & 89.128 & 2.276\% \\ \hline
road_central 79.774 & 88.988 & 11.55\% \\ \hline
hugetrace-00010 66.842 & 84.036 & 25.72\% \\ \hline
road_central 70.548 & 96.228 & 36.4\% \\ \hline
hugetrace-00010 92.602 & 91.877 & 0.7837\% \\ \hline
r

# Dynamic CSR 2D-Partitioning SpMV

In [22]:
csr_dyn_2d_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
m_size = []

csr_2d_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_CSR_2D_SpMV.csv")
csr_2d_spmv = csr_2d_spmv[~csr_2d_spmv['Name'].str.contains("rmat")].reset_index(drop=True)
csr_2d_spmv = csr_2d_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index()
csr_2d_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
csr_2d_spmv = csr_2d_spmv.reset_index(drop=True)
csr_2d_spmv = csr_2d_spmv.sort_values(by=["AvgRow", "NonZeroPerRow"]).reset_index(drop=True)

density = []
min_time_factor = 1.0
if min(csr_2d_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(csr_2d_spmv["AvgTime"])
for i,r in csr_2d_spmv.iterrows():
    csr_2d_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["NNZ"])/float(r["AvgRow"] * r["AvgRow"]))
csr_2d_spmv["Density"] = density

column = "AvgTime"
csr_2d_train_data = csr_2d_spmv[~csr_2d_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
csr_2d_test_data = csr_2d_spmv[csr_2d_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

csr_2d_train = csr_2d_train_data[["Nodes", "nProcess", "AvgRow", "NNZ", "NonZeroPerRow", "AvgNPRSD", "Density"]].to_numpy()

csr_2d_test = csr_2d_test_data[["Nodes", "nProcess", "AvgRow", "NNZ", "NonZeroPerRow", "AvgNPRSD", "Density"]].to_numpy()

csr_2d_X = np.concatenate((csr_2d_train, csr_2d_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
csr_2d_X = scaler.fit_transform(csr_2d_X)
X = csr_2d_X[0:len(csr_2d_train)]
csr_2d_test_X = csr_2d_X[len(csr_2d_train):]
train_y = np.array(csr_2d_train_data[column])
test_y = np.array(csr_2d_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,6e4, num=50):
#     for e in np.linspace(0.0001, 0.9, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:{'C': 35518.36734693877, 'epsilon': 0.37900526315789473}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(csr_2d_test_X)
max_err = 0
avg_err = 0
for i,row in csr_2d_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], "&", '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")           
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRow"])
    nnz_r.append(row["NonZeroPerRow"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("CSR 2D Partition")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/csr_2d_test_data.shape[0])
csr_dyn_2d_pred["Name"] = mat
csr_dyn_2d_pred["Model"] = spmv_model
csr_dyn_2d_pred["Nodes"] = node
csr_dyn_2d_pred["Process"] = prcs
csr_dyn_2d_pred["Matrix Size"] = m_size
csr_dyn_2d_pred["Avg Row"] = rows
csr_dyn_2d_pred["Nonzero per Row"] = nnz_r
csr_dyn_2d_pred["Actual Time"] = a_time
csr_dyn_2d_pred["Predicted Time"] = p_time
csr_dyn_2d_pred["Error"] = err

M6 & 3.9667 & 4.526 & 14.1\% \\ \hline
333SP & 4.2354 & 4.796 & 13.23\% \\ \hline
M6 & 4.3555 & 5.2126 & 19.68\% \\ \hline
AS365 & 4.3713 & 4.9077 & 12.27\% \\ \hline
333SP & 4.7391 & 5.5217 & 16.51\% \\ \hline
AS365 & 4.6825 & 5.6496 & 20.65\% \\ \hline
NLR & 4.8133 & 5.3818 & 11.81\% \\ \hline
M6 & 5.2719 & 6.3175 & 19.83\% \\ \hline
NLR & 5.4101 & 6.1924 & 14.46\% \\ \hline
333SP & 5.7013 & 6.6941 & 17.41\% \\ \hline
M6 & 6.7785 & 6.7905 & 0.1779\% \\ \hline
AS365 & 5.8434 & 6.85 & 17.23\% \\ \hline
333SP & 7.2295 & 7.1955 & 0.4705\% \\ \hline
AS365 & 7.4091 & 7.3634 & 0.6175\% \\ \hline
NLR & 6.4835 & 7.5121 & 15.87\% \\ \hline
NLR & 8.2244 & 8.076 & 1.805\% \\ \hline
hugetrace-00010 & 13.59 & 13.92 & 2.428\% \\ \hline
hugetrace-00010 & 17.388 & 15.86 & 8.787\% \\ \hline
road_central & 15.307 & 15.855 & 3.577\% \\ \hline
hugetrace-00010 & 20.399 & 18.995 & 6.884\% \\ \hline
road_central & 19.163 & 18.028 & 5.927\% \\ \hline
hugetrace-00010 & 23.031 & 20.234 & 12.15\% \\ \hline
road

# Dynamic COO 2D-Partitioning SpMV

In [12]:
coo_dyn_2d_pred = pd.DataFrame()
mat = [] 
node = []
prcs = []
rows = []
nnz_r = []
a_time = []
p_time = []
err = []
spmv_model = []
m_size = []
coo_2d_spmv = pd.read_csv("../../SpMV_Model/src/SkylakeResults/MPI_COO_2D_SpMV.csv")
coo_2d_spmv = coo_2d_spmv[~coo_2d_spmv['Name'].str.contains("rmat")].reset_index(drop=True)
coo_2d_spmv = coo_2d_spmv.groupby(["Name", "Nodes", "nProcess", "DataType"]).mean().reset_index().sort_values(by="Name").reset_index()
coo_2d_spmv.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
coo_2d_spmv = coo_2d_spmv.reset_index(drop=True)
coo_2d_spmv = coo_2d_spmv.sort_values(by=["AvgRow", "NonZeroPerRow"]).reset_index(drop=True)

density = []
min_time_factor = 1.0
if min(coo_2d_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(coo_2d_spmv["AvgTime"])
for i,r in coo_2d_spmv.iterrows():
    coo_2d_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]
    density.append(float(r["NNZ"])/float(r["AvgRow"] * r["AvgRow"]))
coo_2d_spmv["Density"] = density

column = "AvgTime"
coo_2d_train_data = coo_2d_spmv[~coo_2d_spmv["Name"].isin(test_matrices)].reset_index(drop=True)
coo_2d_test_data = coo_2d_spmv[coo_2d_spmv["Name"].isin(test_matrices)].reset_index(drop=True)

coo_2d_train = coo_2d_train_data[["Nodes", "nProcess", "AvgRow", "NNZ", "NonZeroPerRow", "AvgNPRSD", "Density"]].to_numpy()

coo_2d_test = coo_2d_test_data[["Nodes", "nProcess", "AvgRow", "NNZ", "NonZeroPerRow", "AvgNPRSD", "Density"]].to_numpy()

coo_2d_X = np.concatenate((coo_2d_train, coo_2d_test), axis=0)
scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
coo_2d_X = scaler.fit_transform(coo_2d_X)
X = coo_2d_X[0:len(coo_2d_train)]
coo_2d_test_X = coo_2d_X[len(coo_2d_train):]
train_y = np.array(coo_2d_train_data[column])
test_y = np.array(coo_2d_test_data[column])

scores = []
best = 0
# c=20e4
# e=0.001
# for c in np.linspace(2e1,5e5, num=50):
#     for e in np.linspace(0.0001, 0.1, 20):
#         best_svr = SVR(kernel='poly', C=c, gamma='auto', degree=3, epsilon=e, coef0=1)
#         cv = KFold(n_splits=5, random_state=42, shuffle=True)
#         for train_index, test_index in cv.split(X):
#             X_train, X_test, y_train, y_test = X[train_index], X[test_index], train_y[train_index], train_y[test_index]
#             best_svr.fit(X_train, y_train)
#             scores.append(best_svr.score(X_test, np.array(y_test)))
#             if best < np.mean(scores):
#                 best = np.mean(scores)
#                 best_param = {"C": c, "epsilon": e}
best_param:  {'C': 40834.69387755102, 'epsilon': 0.1}
svr = SVR(kernel='poly', C=best_param['C'], gamma='auto', degree=3, epsilon=best_param['epsilon'], coef0=1)
svr.fit(X, train_y)
y_pred = svr.predict(coo_2d_test_X)

max_err = 0
avg_err = 0
for i,row in coo_2d_test_data.iterrows():
#     print(y[index])
#     g = row["Name"].split("_")
    print(row["Name"], "&", '{0:.5g}'.format(test_y[i]/min_time_factor),"&",'{0:.5g}'.format(y_pred[i]/min_time_factor), "&", 
          '{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])+"\\%", "\\\ \hline")           
    if max_err < abs(test_y[i]-y_pred[i])*100/test_y[i]:
        max_err = abs(test_y[i]-y_pred[i])*100/test_y[i]
    avg_err += abs(test_y[i]-y_pred[i])*100/test_y[i]
    mat.append(row["Name"])
    node.append(row["Nodes"])
    prcs.append(row["nProcess"])
    rows.append(row["AvgRow"])
    nnz_r.append(row["NonZeroPerRow"])
    a_time.append(float('{0:.5g}'.format(test_y[i]/min_time_factor)))
    p_time.append(float('{0:.5g}'.format(y_pred[i]/min_time_factor)))
    err.append(float('{0:.4g}'.format(abs(test_y[i]-y_pred[i])*100/test_y[i])))
    spmv_model.append("COO 2D Partition")
    m_size.append(row["MatrixSize"])
# omp_data.groupby("Threads").count()
print("best param: ", best_param)
print("max error: ", max_err, " avg error: ", avg_err/coo_2d_test_data.shape[0])
coo_dyn_2d_pred["Name"] = mat
coo_dyn_2d_pred["Model"] = spmv_model
coo_dyn_2d_pred["Nodes"] = node
coo_dyn_2d_pred["Process"] = prcs
coo_dyn_2d_pred["Matrix Size"] = m_size
coo_dyn_2d_pred["Avg Row"] = rows
coo_dyn_2d_pred["Nonzero per Row"] = nnz_r
coo_dyn_2d_pred["Actual Time"] = a_time
coo_dyn_2d_pred["Predicted Time"] = p_time
coo_dyn_2d_pred["Error"] = err

M6 & 3.3088 & 3.2745 & 1.036\% \\ \hline
333SP & 3.3746 & 3.4688 & 2.793\% \\ \hline
M6 & 3.4268 & 3.5825 & 4.546\% \\ \hline
AS365 & 3.6177 & 3.5493 & 1.891\% \\ \hline
333SP & 3.4904 & 3.7971 & 8.79\% \\ \hline
AS365 & 3.6481 & 3.886 & 6.521\% \\ \hline
NLR & 3.9329 & 3.8905 & 1.08\% \\ \hline
M6 & 3.9087 & 4.2978 & 9.954\% \\ \hline
NLR & 4.0115 & 4.2632 & 6.274\% \\ \hline
333SP & 4.1644 & 4.5608 & 9.518\% \\ \hline
M6 & 5.044 & 4.7886 & 5.064\% \\ \hline
AS365 & 4.2777 & 4.6697 & 9.166\% \\ \hline
333SP & 5.3999 & 5.0817 & 5.893\% \\ \hline
AS365 & 5.5348 & 5.2033 & 5.99\% \\ \hline
NLR & 4.7804 & 5.1328 & 7.372\% \\ \hline
NLR & 6.1182 & 5.7201 & 6.508\% \\ \hline
hugetrace-00010 & 10.213 & 10.257 & 0.4314\% \\ \hline
hugetrace-00010 & 11.378 & 11.22 & 1.394\% \\ \hline
road_central & 11.586 & 11.754 & 1.45\% \\ \hline
hugetrace-00010 & 13.165 & 13.428 & 1.997\% \\ \hline
road_central & 12.835 & 12.839 & 0.03065\% \\ \hline
hugetrace-00010 & 17.099 & 14.81 & 13.39\% \\ \hline
roa

# Overall Evalution

In [28]:
pred = pd.concat([csr_dyn_lk_spmv_pred, coo_dyn_lk_spmv_pred, csr_dyn_gk_spmv_pred, coo_dyn_gk_spmv_pred, csr_dyn_2d_pred])
pred = pred.reset_index(drop=True)

In [29]:
pred = pred.sort_values(by=["Name"]).reset_index(drop=True)
models = ["CSR LK-SPMV", "COO LK-SPMV", "CSR GK-SPMV", "COO GK-SPMV", "CSR 2D Partition"]
print("\multirow{2}{*}{Name} & \multirow{2}{*}{Nodes} & \multirow{2}{*}{Processes} & \multicolumn{5}{c |} {Error\%} \\\\ \cline{4-8}")
print(" &  &  & CSR L1DR & COO L1DRV & CSR G1DR & COO G1DR & CSR 2DU \\\\ \hline")
for g,g_data in pred.groupby("Name"):
    for n,n_data in g_data.groupby("Nodes"):
        for p,p_data in n_data.groupby("Process"):
            p_min = p_data["Predicted Time"].min()
            pr_best_a = p_data[p_data["Predicted Time"] == p_min]["Model"].iloc[0]
            a_min = p_data["Actual Time"].min()
            ac_best_a = p_data[p_data["Actual Time"] == a_min]["Model"].iloc[0]
#             print(pr_best_a, " ", ac_best_a)
            print(g, ' & ', n, ' & ', p, end=' & ')
            i=1
            for m in models: #,m_data in p_data.groupby("Model"):
                m_data = p_data[p_data["Model"] == m].reset_index(drop=True)
                if m==ac_best_a:
                    if pr_best_a==ac_best_a and pr_best_a == m:
                        print("\cellcolor{green!25} ", float('{0:.3g}'.format(m_data["Error"].iloc[0])), end=' & ' if i<len(p_data) else ' \\\\ \hline')
                    else:
                        print("\cellcolor{blue!25} ", float('{0:.3g}'.format(m_data["Error"].iloc[0])), end=' & ' if i<len(p_data) else ' \\\\ \hline')
                elif m==pr_best_a:
                    print("\cellcolor{red!25} ", float('{0:.3g}'.format(m_data["Error"].iloc[0])), end=' & ' if i<len(p_data) else ' \\\\ \hline')
                else:
                    print(float('{0:.3g}'.format(m_data["Error"].iloc[0])), end=' & ' if i<len(p_data) else ' \\\\ \hline')
                i+=1
            print()
    
# pred

\multirow{2}{*}{Name} & \multirow{2}{*}{Nodes} & \multirow{2}{*}{Processes} & \multicolumn{5}{c |} {Error\%} \\ \cline{4-8}
 &  &  & CSR L1DR & COO L1DRV & CSR G1DR & COO G1DR & CSR 2DU \\ \hline
333SP  &  4  &  144 & \cellcolor{green!25}  15.8 & 15.5 & 3.11 & 2.24 & 0.47 \\ \hline
333SP  &  5  &  169 & \cellcolor{green!25}  17.6 & 18.7 & 11.9 & 6.55 & 17.4 \\ \hline
333SP  &  7  &  225 & \cellcolor{green!25}  20.3 & 20.9 & 8.65 & 6.21 & 16.5 \\ \hline
333SP  &  8  &  256 & \cellcolor{green!25}  24.6 & 22.6 & 5.36 & 3.16 & 13.2 \\ \hline
AS365  &  4  &  144 & \cellcolor{green!25}  20.2 & 19.1 & 2.98 & 2.15 & 0.618 \\ \hline
AS365  &  5  &  169 & \cellcolor{green!25}  19.7 & 17.7 & 11.5 & 6.58 & 17.2 \\ \hline
AS365  &  7  &  225 & \cellcolor{green!25}  23.2 & 20.0 & 9.61 & 5.71 & 20.6 \\ \hline
AS365  &  8  &  256 & \cellcolor{red!25}  22.8 & \cellcolor{blue!25}  18.1 & 5.38 & 3.35 & 12.3 \\ \hline
M6  &  4  &  144 & \cellcolor{green!25}  20.7 & 18.8 & 3.22 & 3.5 & 0.178 \\ \hline
M6  

In [14]:
for g,g_data in pred.groupby("Name"):
    for n,n_data in g_data.groupby("Nodes"):
        for p,p_data in n_data.groupby("Process"):
            print(g, p, end=' ')
            for m,m_data in p_data.groupby("Model"):
                print(m, m_data["Actual Time"].iloc[0], end=" ")
            print()

AS365 144 COO GK-SPMV 33.306 COO LK-SPMV 1.3544 CSR 2D Partition 7.4091 CSR GK-SPMV 26.738 CSR LK-SPMV 1.3116 
AS365 169 COO GK-SPMV 27.348 COO LK-SPMV 1.2087 CSR 2D Partition 5.8434 CSR GK-SPMV 20.554 CSR LK-SPMV 1.1269 
AS365 225 COO GK-SPMV 25.025 COO LK-SPMV 0.99845 CSR 2D Partition 4.6825 CSR GK-SPMV 19.748 CSR LK-SPMV 0.90991 
AS365 256 COO GK-SPMV 27.8 COO LK-SPMV 0.8581 CSR 2D Partition 4.3713 CSR GK-SPMV 24.177 CSR LK-SPMV 0.8264 
NACA0015 144 COO GK-SPMV 9.3246 COO LK-SPMV 0.36924 CSR 2D Partition 1.9453 CSR GK-SPMV 7.5914 CSR LK-SPMV 0.38137 
NACA0015 169 COO GK-SPMV 7.5423 COO LK-SPMV 0.36068 CSR 2D Partition 2.0708 CSR GK-SPMV 5.6271 CSR LK-SPMV 0.34119 
NACA0015 225 COO GK-SPMV 6.8483 COO LK-SPMV 0.28833 CSR 2D Partition 1.3627 CSR GK-SPMV 5.4022 CSR LK-SPMV 0.31422 
NACA0015 256 COO GK-SPMV 7.696 COO LK-SPMV 0.3051 CSR 2D Partition 1.2435 CSR GK-SPMV 6.7073 CSR LK-SPMV 0.30221 
NLR 144 COO GK-SPMV 36.468 COO LK-SPMV 1.4252 CSR 2D Partition 8.2244 CSR GK-SPMV 29.247 CSR L

In [32]:
def get_optimal(_test_matrices, _csr_kway_pred, _coo_kway_pred, _csr_local_kway_pred, _coo_local_kway_pred, _csr_2d_pred):
    _mat = []
    _p = []
    _algo = []
    _ralgo = []
    _csr_g1dr = []
    _coo_g1dr = []
    _csr_l1dr = []
    _coo_l1dr = []
    _csr_2d = []
    _coo_2d = []
    _pr_csr_g1dr = []
    _pr_coo_g1dr = []
    _pr_csr_l1dr = []
    _pr_coo_l1dr = []
    _pr_csr_2d = []
    _pr_coo_2d = []
    _right = []
    _ac = []
    _pr = []
    _optimal = pd.DataFrame()
    for m in _test_matrices:
        for p in [144, 169, 225]:
            _p.append(p)
            _mat.append(m)
#             print(p, m)
            al1 = _csr_kway_pred[(_csr_kway_pred["Name"] == m) & (_csr_kway_pred["Process"] == p)].reset_index(drop=True).iloc[0]
            csr_gk = al1["Predicted Time"]
            min_t = csr_gk;
            min_a = "CSR G1DR SpMV"
            r_a = "CSR G1DR SpMV"
            r_t = al1["Actual Time"]
            _csr_g1dr.append(al1["Actual Time"])
            _pr_csr_g1dr.append(al1["Predicted Time"])
            al5 = _csr_2d_pred[(_csr_2d_pred["Name"] == m) & (_csr_2d_pred["Process"] == p)].reset_index(drop=True).iloc[0]
            csr_2d = al5["Predicted Time"]
            _csr_2d.append(al5["Actual Time"])
            _pr_csr_2d.append(al5["Predicted Time"])
            if min_t > csr_2d:
                min_t = csr_2d
                min_a = "CSR 2DU"
            if r_t > al5["Actual Time"]:
                r_t = al5["Actual Time"]
                r_a = "CSR 2DU"
            al2 = _coo_kway_pred[(_coo_kway_pred["Name"] == m) & (_coo_kway_pred["Process"] == p)].reset_index(drop=True).iloc[0]
            coo_gk = al2["Predicted Time"]
            _coo_g1dr.append(al2["Actual Time"])
            _pr_coo_g1dr.append(al2["Predicted Time"])
            if min_t > coo_gk:
                min_t = coo_gk
                min_a = "COO G1DR SpMV"
            if r_t > al2["Actual Time"]:
                r_t = al2["Actual Time"]
                r_a = "COO G1DR SpMV"
            al3 = _csr_local_kway_pred[(_csr_local_kway_pred["Name"] == m) & (_csr_local_kway_pred["Process"] == p)].reset_index(drop=True).iloc[0]
            csr_lk = al3["Predicted Time"]
            _csr_l1dr.append(al3["Actual Time"])
            _pr_csr_l1dr.append(al3["Predicted Time"])
            if min_t > csr_lk:
                min_t = csr_lk
                min_a = "CSR L1DR SpMV"
            if r_t > al3["Actual Time"]:
                r_t = al3["Actual Time"]
                r_a = "CSR L1DR SpMV"
            al4 = _coo_local_kway_pred[(_coo_local_kway_pred["Name"] == m) & (_coo_local_kway_pred["Process"] == p)].reset_index(drop=True).iloc[0]
            coo_lk = al4["Predicted Time"]
            _coo_l1dr.append(al4["Actual Time"])
            _pr_coo_l1dr.append(al4["Predicted Time"])
            if min_t > coo_lk:
                min_t = coo_lk
                min_a = "COO L1DR SpMV"
            if r_t > al4["Actual Time"]:
                r_t = al4["Actual Time"]
                r_a = "COO L1DR SpMV"
            _algo.append(min_a)
            _ralgo.append(r_a)
            _ac.append(r_t)
            _pr.append(min_t)
            if min_a == r_a:
                _right.append(1)
            else:
                _right.append(0)
    #         print(p, " algo: ", min_a, " time:", min_t, " csr lk:", csr_lk, " csr gk:", csr_gk, " csr 2d:", csr_2d, " coo lk:", coo_lk, " coo gk:", coo_gk, " coo 2d:", coo_2d)
    _optimal["Matrices"] = _mat
    _optimal["Processes"] = _p
    _optimal["Predicted Best Strategy"] = _algo
    _optimal["Actual Best Strategy"] = _ralgo
    _optimal["R/W"] = _right
    _optimal["CSR L1DR SpMV"] = _csr_l1dr
    _optimal["CSR L1DR SpMV Predicted"] = _pr_csr_l1dr
    _optimal["COO L1DR SpMV"] = _coo_l1dr
    _optimal["COO L1DR SpMV Predicted"] = _pr_coo_l1dr
    _optimal["CSR G1DR SpMV"] = _csr_g1dr
    _optimal["CSR G1DR SpMV Predicted"] = _pr_csr_g1dr
    _optimal["COO G1DR SpMV"] = _coo_g1dr
    _optimal["COO G1DR SpMV Predicted"] = _pr_coo_g1dr
    _optimal["CSR 2DU"] = _csr_2d
    _optimal["CSR 2DU Predicted"] = _pr_csr_2d
    _optimal["Predicted Time"] = _ac
    _optimal["Actual Time"] = _pr
    _optimal[_optimal["Processes"] == 225]
    return _optimal

In [34]:
optimal = get_optimal(test_matrices, csr_dyn_gk_spmv_pred, coo_dyn_gk_spmv_pred, csr_dyn_lk_spmv_pred, coo_dyn_lk_spmv_pred, csr_dyn_2d_pred, coo_dyn_2d_pred)
optimal

TypeError: get_optimal() takes 6 positional arguments but 7 were given

In [17]:
print("performance of all")
# for i,r in optimal.iterrows():
#     print(r["Matrices"], " & \\backslashbox{", round(r["CSR L1DR SpMV"], 2),"}{",round(r["CSR L1DR SpMV Predicted"], 2), "} & \\backslashbox{", round(r["COO L1DR SpMV"], 2), "}{", round(r["COO L1DR SpMV Predicted"], 2), "} & \\backslashbox{", round(r["CSR G1DR SpMV"], 2), "}{", round(r["CSR G1DR SpMV Predicted"], 2), "} & \\backslashbox{", round(r["COO G1DR SpMV"], 2), "}{", round(r["COO G1DR SpMV Predicted"], 2), "} & \\backslashbox{", round(r["CSR 2DU"], 2), "}{", round(r["CSR 2DU Predicted"], 2), "} & \\backslashbox{", round(r["COO 2DU"], 2), "}{", round(r["COO 2DU Predicted"], 2), "} \\\ \hline")
for i,r in optimal.sort_values(by="Matrices").reset_index().iterrows():
    print(r["Matrices"], " & ", r["Processes"], " & ", round(r["CSR L1DR SpMV"], 2)," & ",round(r["CSR L1DR SpMV Predicted"], 2), " & ", round(r["COO L1DR SpMV"], 2), " & ", round(r["COO L1DR SpMV Predicted"], 2), " & ", round(r["CSR G1DR SpMV"], 2), " & ", round(r["CSR G1DR SpMV Predicted"], 2), " & ", round(r["COO G1DR SpMV"], 2), " & ", round(r["COO G1DR SpMV Predicted"], 2), " & ", round(r["CSR 2DU"], 2), " & ", round(r["CSR 2DU Predicted"], 2), " \\\ \hline")

print("Best Alforithm")
for i,r in optimal.iterrows():
    print(r["Matrices"], " & ", r["Predicted Best Strategy"], " & ", r["Actual Best Strategy"], " & ", r["Predicted Time"], " & ", r["Actual Time"], " & ", r["R/W"], " \\\ \hline")

performance of all
AS365  &  144  &  1.31  &  1.07  &  1.35  &  1.13  &  26.74  &  25.94  &  33.31  &  32.62  &  7.41  &  7.24  \\ \hline
AS365  &  169  &  1.13  &  0.93  &  1.21  &  0.98  &  20.55  &  22.88  &  27.35  &  29.01  &  5.84  &  6.78  \\ \hline
AS365  &  225  &  0.91  &  0.73  &  1.0  &  0.77  &  19.75  &  21.53  &  25.02  &  26.44  &  4.68  &  5.64  \\ \hline
NACA0015  &  144  &  0.38  &  0.33  &  0.37  &  0.35  &  7.59  &  7.27  &  9.32  &  9.13  &  1.95  &  2.02  \\ \hline
NACA0015  &  169  &  0.34  &  0.29  &  0.36  &  0.31  &  5.63  &  6.02  &  7.54  &  7.69  &  2.07  &  1.89  \\ \hline
NACA0015  &  225  &  0.31  &  0.24  &  0.29  &  0.25  &  5.4  &  5.71  &  6.85  &  7.01  &  1.36  &  1.59  \\ \hline
NLR  &  144  &  1.4  &  1.17  &  1.43  &  1.24  &  29.25  &  28.41  &  36.47  &  35.75  &  8.22  &  7.94  \\ \hline
NLR  &  169  &  1.26  &  1.01  &  1.29  &  1.07  &  22.43  &  25.11  &  29.96  &  31.85  &  6.48  &  7.44  \\ \hline
NLR  &  225  &  0.99  &  0.8  &  1.01  

In [158]:
csr_2d_spmv[csr_2d_spmv["Name"] == "AS365"]
csr_dyn_2d_pred
min_time_factor = 1.0
if min(csr_dyn_lk_spmv["AvgTime"]) < 1.0:
    min_time_factor = 1/min(csr_dyn_lk_spmv["AvgTime"])
for i,r in csr_dyn_lk_spmv.iterrows():
    csr_dyn_lk_spmv.at[i, "AvgTime"] = min_time_factor*r["AvgTime"]

,Name,Model,Nodes,Process,Matrix Size,Avg Row,Nonzero per Row,Actual Time,Predicted Time,Error
0,NACA0015,CSR 2D Partition,5,225,1039183,69279,0.399649,0.053592,0.049382,7.85600
1,delaunay_n20,CSR 2D Partition,5,225,1048576,69906,0.399989,0.054246,0.049890,8.02900
2,NACA0015,CSR 2D Partition,4,169,1039183,79938,0.461129,0.061411,0.040047,34.79000
3,delaunay_n20,CSR 2D Partition,4,169,1048576,80660,0.461531,0.061718,0.040716,34.03000
4,NACA0015,CSR 2D Partition,3,144,1039183,86599,0.499560,0.067617,0.052022,23.06000
5,delaunay_n20,CSR 2D Partition,3,144,1048576,87382,0.499990,0.068403,0.052821,22.78000
6,AS365,CSR 2D Partition,5,225,3799275,253285,0.398956,0.165380,0.192590,16.46000
7,NLR,CSR 2D Partition,5,225,4163763,277585,0.399893,0.195050,0.211840,8.60800
8,AS365,CSR 2D Partition,4,169,3799275,292252,0.460334,0.212730,0.222930,4.79200
9,AS365,CSR 2D Partition,3,144,3799275,316607,0.498694,0.269720,0.269940,0.08134
